# Lectura y balanceo de datos

## Lectura

In [48]:
#Paquetes
import tensorflow as tf
import numpy as np

#Extracción de datos
lote = 32 #Numero de muestras con el que se trabaja hasta que se actualizan los parametros
data_training = tf.keras.preprocessing.text_dataset_from_directory( #Se necesita TensorFlow version 2.3.0 o superior
    "aclImdb/train", #Ruta de los datos
    batch_size=lote, #Tamaño del lote
    validation_split=0.2, #Proporcion de muestras de validacion
    subset="training", #Tipo de subset
    seed=1337, #Semilla desde la que empiezo a coger muestras
)
data_validation = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=lote,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
data_test = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=lote 
)

#Número de lotes
print(
    "Número de lotes de entrenamiento: %d"
    % tf.data.experimental.cardinality(data_training)
)

print(
    "Número de lotes de validación: %d" % tf.data.experimental.cardinality(data_validation)
)

print(
    "Número de lotes de prueba: %d"
    % tf.data.experimental.cardinality(data_test))

In [49]:
#Bucle para extraer los valores
for texto, etiqueta in data_training.take(1):
    for i in range(5):
        print('Texto: ',texto.numpy()[i])
        print('Etiqueta: ',etiqueta.numpy()[i])

# Preparación del texto

In [31]:
#Paquetes
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

# Creamos una función para eliminar las / de nuestro texto, y poner todas las letras en minúsculas
def preparacion(texto):
    texto_minusculas = tf.strings.lower(texto)
    sin_barras = tf.strings.regex_replace(texto_minusculas, "<br />", " ")
    return tf.strings.regex_replace(
        sin_barras, "[%s]" % re.escape(string.punctuation), "")


# Definimos la vectorización.
caracteristicas = 20000 #Número máximo de tokens de la capa (con cuantas palabras trabaja)
long_seq = 500 #Longitud máxima de cada secuencia

# Definimos la capa de vectorizacion
capa_vectorizacion = TextVectorization(
    #Definimos los valores
    standardize=preparacion,
    max_tokens=caracteristicas,
    output_mode="int", #El vector que sale es de enteros
    output_sequence_length=long_seq,
)

texto = data_training.map(lambda x, y:x ) #Me quedo con solo el texto, sin etiquetas
capa_vectorizacion.adapt(texto) #Adapto la capa de vectorizacion al texto


## Modelo predictivo

In [32]:
#Métricas de evaluación
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [50]:
#Importamos el paquete de las capas neuronales
from tensorflow.keras import layers

#Primero vectorizo el texto de entrada
inputs = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')#La forma de entrada es un array de texto
x = capa_vectorizacion(inputs)

# Después generamos la capa de incrustación - Sirve para vectorizar las características en el mapa
caracteristicas = 20000 #Número máximo de tokens de la capa (con cuantas palabras trabaja)
incrustacion_dim = 128 #Dimensiones de la capa de incrustacion 
x = layers.Embedding(caracteristicas + 1, incrustacion_dim)(x) #Se genera la capa de incrustacion 

#Despues añadimos una capa de dropout para evitar el sobreentrenamiento
x = layers.Dropout(0.5)(x)

#Generamos ahora las capas convolucionales
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x) #128 filtros y 7 unidades el tamaño 
#nucleo, sin padding (sin añadir ceros al final del vector) con la funcion de activacion relu y con una longitud
#de 3 unidades en la convolucion
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)

#Genero ahora una capa de agrupacion global por maximos en una dimension
x = layers.GlobalMaxPooling1D()(x)

# Finalmente añadimos una capa densa con un nuevo dropout para terminar el proceso
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# Genero la capa densa final, con una unica neurona que me el resultado de si el texto es 1 o 0
# Usamos en este caso una funcion de activacion sigmoidea
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

#Generamos todo el modelo con las capas que hemos definido
model = tf.keras.Model(inputs, predictions)

#Resumen del modelo elaborado
model.summary()

# Compilamos la red, siendo la funcion de coste la entropia binaria cruzada, el optimizador adam y las metricas explicadas
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy',f1_m,precision_m, recall_m])

In [51]:
#Especifico cada cuantos grupos actualizo los parametros de la red
subgrupos = 3

# Ajustamos el modelo a los conjuntos de entrenamiento y validacion
model.fit(data_training, validation_data=data_validation, epochs=subgrupos)

In [52]:
#Evaluamos el modelo
model.evaluate(data_test)

In [53]:
#Finalmente se podría realizar una prediccion
model.predict(['I think this text is so bad'])